<a href="https://colab.research.google.com/github/Yashasvi-S-Khade/CV/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install streamlit_lottie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
%%writefile index.py

import requests
import streamlit as st
from streamlit_lottie import st_lottie
from PIL import Image
import tensorflow as tf
import numpy as np
from inceptionV3 import deep_dream_inception
from resNet50 import deep_dream_resNet50
from mobileNet import deep_dream_mobileNet


st.set_page_config(page_title="Deep Dream", page_icon=":nazar_amulet:", layout="wide")

def get_layer_info(model, layer_number):
    """
    Retrieve information about the selected layer.
    """
    layer = model.layers[layer_number]
    layer_name = layer.name
    layer_type = type(layer).__name__
    input_shape = layer.input_shape
    output_shape = layer.output_shape

    return {
        "Layer Name": layer_name,
        "Layer Type": layer_type,
        "Input Shape": input_shape,
        "Output Shape": output_shape
    }


def load_lottieurl(url):
    r = requests.get(url)
    if r.status_code != 200:
        return None
    return r.json()

lottie_dreaming = load_lottieurl("https://lottie.host/b5551b7e-d91f-4bb1-b209-651ec55eb22e/mtLlUTl3iO.json")

#-------HEADER SECTION --------
with st.container():
    st.subheader("Hello User :wave:")
    st.title("Deep Dreaming")
    st.write("The future belongs to those who believe in the beauty of their DREAMS")
    st.write("[Learn More >](https://medium.com/hashworks/deep-dreaming-with-deep-learning-487835ebf315)")


with st.container():
    st.write("---")
    left_column, right_column = st.columns(2)
    with left_column:
        st.header("About")
        st.write("##")
        st.write(
            """
            Deep dreaming involves the generation of machine hallucinated images.
            These wildly imaginative visuals are generated by a neural network that is actually a series of statistical learning models, powered by deceptively simple algorithms that are modelled after evolutionary processes.
            Researchers have “trained” these networks by feeding them millions of images and gradually adjusting the network’s parameters until it gives the desired classification.

            The outputs are not just meaningless patterns of neurons,
            but depend on previous learning that the network undergoes representing data “attractors” — where some random neurons start to fire,
            the weighted connections representing real output rapidly come to dominate the overall pattern of activity in the network, resulting in the pattern corresponding to a particular input.
            """
        )
        st.write("[Google Research >](https://blog.research.google/2015/06/inceptionism-going-deeper-into-neural.html)")
        with right_column:
            st_lottie(lottie_dreaming, height=400, key="dreaming")
            # Add a button
def first_page():

    if st.button("Go to Image Processing Page"):
       st.session_state.page = "Image Processing Page"

# Function to define the second page
def image_processing_page():
    st.title("Image Processing Page")

    uploaded_image = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])

    if uploaded_image is not None:
        image = Image.open(uploaded_image)

        # Display uploaded image in the center with width 1/3 of screen width
        st.image(image, caption="Uploaded Image", use_column_width=True)

        # Create a dropdown menu to select the model
        selected_model = st.selectbox("Select a Model", ["InceptionV3", "ResNet50","MobileNet"])

        # Load the selected model
        if selected_model == "InceptionV3":
            model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
        elif selected_model == "ResNet50":
            model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
        elif selected_model == "MobileNet":
            model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)


        # Get the layer number from the user
        num_layers = len(model.layers)
        layer_number = st.number_input(f"Enter the layer number (0 to {num_layers-1}): ", 0, num_layers-1, 0)

        # Validate the user input
        if layer_number < 0 or layer_number >= num_layers:
            st.error(f"Invalid layer number. Please enter a number between 0 and {num_layers-1}.")
        else:
            layer_info = get_layer_info(model, layer_number)
            # Display layer information
            st.subheader("Selected Layer Information")
            for key, value in layer_info.items():
                st.write(f"**{key}:** {value}")
            layer_name = model.layers[layer_number].name
            st.write(f"You selected Layer: {layer_name}")

            # Generate the reversed original image from the DeepDream image
            if selected_model == "InceptionV3":
                original_image = deep_dream_inception(model, image, layer_name)
            elif selected_model == "ResNet50":
                original_image = deep_dream_resNet50(model, image, layer_name)
            elif selected_model == "MobileNet":
                original_image = deep_dream_mobileNet(model, image, layer_name)


            # Display original and dream images side by side
            col1, col2 = st.columns(2)
            with col1:
                st.image(image, caption="Original Image", use_column_width=True)
            with col2:
                st.image(original_image, caption="Dream Image", use_column_width=True)



    if st.button("Back to First Page"):
        st.session_state.page = "First Page"


# Set the initial page to "First Page"
if "page" not in st.session_state:
    st.session_state.page = "First Page"

# Define the pages
if st.session_state.page == "First Page":
    first_page()
elif st.session_state.page == "Image Processing Page":
    image_processing_page()

Overwriting index.py


In [ ]:
%%writefile mobileNet.py

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def deep_dream_mobileNet(model, image, layer_name, iterations=5, step_size=0.01, octave_scale=1.5, num_octaves=10):
    img = tf.keras.preprocessing.image.img_to_array(image)
    img = tf.image.resize(img,(224,224))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

    # Reshape the input image to match the expected shape of the model
    img = np.expand_dims(img, axis=0)

    # Create a model that outputs the activation of the specified layer
    layer_output = model.get_layer(layer_name).output
    dream_model = tf.keras.Model(model.input, layer_output)

    # Generate the DeepDream image
    original_shape = img.shape[1:-1]
    img = tf.image.resize(img, original_shape)

    for _ in range(num_octaves):
        # Generate details from the upsampled image
        img_details = tf.Variable(img)
        for _ in range(iterations):
            with tf.GradientTape() as tape:
                tape.watch(img_details)
                outputs = dream_model(img_details)
                loss = tf.reduce_mean(outputs)

            gradients = tape.gradient(loss, img_details)
            gradients /= tf.math.reduce_std(gradients) + 1e-8
            img_details = tf.Variable(img_details + gradients * step_size)

            img_details = tf.clip_by_value(img_details, -1, 1)

        # Upscale the image for the next octave
        img = tf.image.resize(img_details, original_shape)
        step_size *= octave_scale

    img = tf.clip_by_value(img, -1, 1)
    img = tf.keras.preprocessing.image.array_to_img(img[0])
    return img

Writing mobileNet.py


In [ ]:
%%writefile resNet50.py


import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def deep_dream_resNet50(model, image, layer_name, iterations=5, step_size=0.01, octave_scale=1.5, num_octaves=10):
    img = tf.keras.preprocessing.image.img_to_array(image)
    #img = tf.image.resize(img,(299,299))
    img = tf.keras.applications.resnet.preprocess_input(img)

    # Reshape the input image to match the expected shape of the model
    img = np.expand_dims(img, axis=0)

    # Create a model that outputs the activation of the specified layer
    layer_output = model.get_layer(layer_name).output
    dream_model = tf.keras.Model(model.input, layer_output)

    # Generate the DeepDream image
    original_shape = img.shape[1:-1]
    img = tf.image.resize(img, original_shape)

    for _ in range(num_octaves):
        # Generate details from the upsampled image
        img_details = tf.Variable(img)
        for _ in range(iterations):
            with tf.GradientTape() as tape:
                tape.watch(img_details)
                outputs = dream_model(img_details)
                loss = tf.reduce_mean(outputs)

            gradients = tape.gradient(loss, img_details)
            gradients /= tf.math.reduce_std(gradients) + 1e-8
            img_details = tf.Variable(img_details + gradients * step_size)

            img_details = tf.clip_by_value(img_details, -1, 1)

        # Upscale the image for the next octave
        img = tf.image.resize(img_details, original_shape)
        step_size *= octave_scale

    img = tf.clip_by_value(img, -1, 1)
    img = tf.keras.preprocessing.image.array_to_img(img[0])
    return img

Writing resNet50.py


In [ ]:
%%writefile inceptionV3.py


import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def deep_dream_inception(model, image, layer_name, iterations=20, step_size=0.01, octave_scale=1.5, num_octaves=3):
    img = tf.keras.preprocessing.image.img_to_array(image)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)

    # Reshape the input image to match the expected shape of the model
    img = np.expand_dims(img, axis=0)

    # Create a model that outputs the activation of the specified layer
    layer_output = model.get_layer(layer_name).output
    dream_model = tf.keras.Model(model.input, layer_output)

    # Generate the DeepDream image
    original_shape = img.shape[1:-1]
    img = tf.image.resize(img, original_shape)

    for _ in range(num_octaves):
        # Generate details from the upsampled image
        img_details = tf.Variable(img)
        for _ in range(iterations):
            with tf.GradientTape() as tape:
                tape.watch(img_details)
                outputs = dream_model(img_details)
                loss = tf.reduce_mean(outputs)

            gradients = tape.gradient(loss, img_details)
            gradients /= tf.math.reduce_std(gradients) + 1e-8
            img_details = tf.Variable(img_details + gradients * step_size)

            img_details = tf.clip_by_value(img_details, -1, 1)

        # Upscale the image for the next octave
        img = tf.image.resize(img_details, original_shape)
        step_size *= octave_scale

    img = tf.clip_by_value(img, -1, 1)
    img = tf.keras.preprocessing.image.array_to_img(img[0])
    return img

# # Load the InceptionV3 model
# model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)

# # Get the number of layers in the model
# num_layers = len(model.layers)

# # Get the layer number from user input
# layer_number = int(input(f"Enter the layer number (0 to {num_layers-1}): "))

# # Validate the user input
# if layer_number < 0 or layer_number >= num_layers:
#     print(f"Invalid layer number. Please enter a number between 0 and {num_layers-1}.")
#     exit()

# # Load an example image
# image_path = 'mona.jpg'
# image = tf.keras.preprocessing.image.load_img(image_path)       #, target_size=(675, 1200)

# # Specify the layer name based on the user input
# layer_name = model.layers[layer_number].name

# # Generate the DeepDream image for the selected layer
# dream_image = deep_dream(model, image, layer_name)

# # Display the original image and the DeepDream image
# # fig, axes = plt.subplots(1, 2)
# # axes[0].imshow(image)
# # axes[0].set_title('Original Image')
# # axes[1].imshow(dream_image)
# # axes[1].set_title('DeepDream Image (Layer: ' + layer_name + ')')
# # plt.show()

# fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Adjust the figsize as needed

# axes[0].imshow(image)
# axes[0].set_title('Original Image')
# axes[1].imshow(dream_image)
# axes[1].set_title('DeepDream Image (Layer: ' + layer_name + ')')

# plt.tight_layout()  # Optional: Adjust the layout for better spacing
# plt.show()
# #dream_image.show();

Writing inceptionV3.py


In [ ]:
!streamlit run index.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.488s
your url is: https://nine-feet-stop.loca.lt
